## Extract relevant information from WHOGMDatabase.csv - from Shaddick et al.

* Find ground monitor locations which have reported PM25 values later than 2015
* Get latest year
* create categorical country code
* add binary classification "is_urban"

Creates two data files:

* `world_pm25.csv`
* `north_america_pm25.csv`


In [1]:
import json
import pandas as pd
import numpy as np


In [2]:
who_clean = pd.read_csv('WHOGMDatabase.csv', encoding = "ISO-8859-1")

In [3]:
who_clean.keys()

Index(['ISO3', 'CountryName', 'Year', 'StationID', 'StationIDOrig',
       'StationIDOldDatabase', 'City', 'CityReverseGeocoded', 'CityGiulia',
       'CityClean', 'Longitude', 'Latitude', 'PM25', 'PM25PercCoverage',
       'PM25Grading', 'PM10', 'PM10PercCoverage', 'PM10Grading',
       'LocationInfo', 'Source', 'MonitorType', 'MonitorTypeOrig', 'PM25Conv',
       'UnspecifiedType', 'WebLink', 'Version', 'WHOStatus', 'WHORegion',
       'WHOIncomeRegion', 'SDG1Region', 'SDG2Region', 'SDG3Region',
       'GBDRegion', 'GBDSuperRegion'],
      dtype='object')

In [15]:
who_clean['GBDSuperRegion'].unique()

array(['South Asia', 'Central Europe, Eastern Europe and Central Asia',
       'High income', 'North Africa / Middle East',
       'Latin America and Caribbean',
       'Southeast Asia, East Asia and Oceania', 'Sub-Saharan Africa'],
      dtype=object)

In [17]:
who_clean['SDG2Region'].unique()

array(['Southern Asia', 'Europe', 'Western Asia',
       'Latin America and the Caribbean', 'Oceania', 'South-eastern Asia',
       'Northern America', 'Eastern Asia', 'Sub-Saharan Africa',
       'Northern Africa', 'Central Asia'], dtype=object)

In [18]:
world = who_clean.drop(columns=['StationIDOrig',
    'StationIDOldDatabase', 'City', 'CityGiulia', 'CityClean',
    'PM25PercCoverage', 'PM25Grading',
    'PM10', 'PM10PercCoverage', 'PM10Grading',
    'LocationInfo', 'Source', 'MonitorTypeOrig', 'PM25Conv',                           
    'UnspecifiedType', 'WebLink', 'Version', 'WHOStatus', 'WHORegion',
    'WHOIncomeRegion', 'SDG1Region', 'SDG3Region',
    'GBDRegion', 'GBDSuperRegion'], axis= 1)

In [19]:
world.keys()

Index(['ISO3', 'CountryName', 'Year', 'StationID', 'CityReverseGeocoded',
       'Longitude', 'Latitude', 'PM25', 'MonitorType', 'SDG2Region'],
      dtype='object')

In [20]:
print(world.shape)
world = world[world['Year']>2015]
print(world.shape)

(65428, 10)
(31454, 10)


In [21]:
world = world.dropna(subset =['PM25'])
world.shape

(20359, 10)

In [22]:
world = world.groupby(['StationID']).apply(lambda x: x.loc[x['Year'].idxmax()], include_groups=False).reset_index(drop=True)
world.shape


(6495, 9)

In [23]:
world['is_urban'] = np.where(world['MonitorType'].isin(['Urban', 'urban', 'Industrial', 'industrial']), 1, 0)
world['country'] = world['ISO3'].astype('category').cat.codes + 1


In [24]:
world

,ISO3,CountryName,Year,CityReverseGeocoded,Longitude,Latitude,PM25,MonitorType,SDG2Region,is_urban,country
0,AFG,Afghanistan,2019,Kabul,69.190510,34.535810,119.773600,Unknown,Southern Asia,0,1
1,ALB,Albania,2016,DurrÃ«s,19.449200,41.319900,14.323250,Traffic,Europe,0,2
2,ALB,Albania,2019,VlorÃ«,19.486200,40.403090,10.315249,Background,Europe,0,2
3,ALB,Albania,2016,KorÃ§Ã«,20.780180,40.625930,28.640652,Background,Europe,0,2
4,AND,Andorra,2017,Les Escaldes,1.539138,42.509694,10.300000,Unknown,Europe,0,3
...,...,...,...,...,...,...,...,...,...,...,...
6490,ZAF,South Africa,2021,Embalenhle,29.112220,-26.551390,16.673530,Industrial,Sub-Saharan Africa,1,99
6491,ZAF,South Africa,2021,Secunda,29.189720,-26.523610,21.289610,Urban,Sub-Saharan Africa,1,99
6492,ZAF,South Africa,2018,Secunda,29.210830,-26.605560,19.273150,Industrial,Sub-Saharan Africa,1,99
6493,ZAF,South Africa,2019,Standerton,29.223470,-26.964080,27.106370,Residential,Sub-Saharan Africa,0,99


In [29]:
import csv
world.to_csv('world_pm25.csv', 
             columns=['Year', 'CityReverseGeocoded', 'Longitude', 'Latitude', 
                      'PM25', 'is_urban', 'country'], 
                     quoting=csv.QUOTE_NONNUMERIC, index=False)

In [26]:
euro_slice = world[world['SDG2Region'] == 'Europe']
euro_slice.shape

(1865, 11)

In [27]:
euro = euro_slice.copy(deep=True).reset_index(drop=True)
euro['country'] = euro['ISO3'].astype('category').cat.codes + 1
euro

,ISO3,CountryName,Year,CityReverseGeocoded,Longitude,Latitude,PM25,MonitorType,SDG2Region,is_urban,country
0,ALB,Albania,2016,DurrÃ«s,19.449200,41.319900,14.323250,Traffic,Europe,0,1
1,ALB,Albania,2019,VlorÃ«,19.486200,40.403090,10.315249,Background,Europe,0,1
2,ALB,Albania,2016,KorÃ§Ã«,20.780180,40.625930,28.640652,Background,Europe,0,1
3,AND,Andorra,2017,Les Escaldes,1.539138,42.509694,10.300000,Unknown,Europe,0,2
4,AUT,Austria,2019,Sankt Michael,14.574722,48.531111,7.248565,Background,Europe,0,3
...,...,...,...,...,...,...,...,...,...,...,...
1860,XKO,Kosovo (under UNSCR 1244/99),2019,NaN,20.532700,42.373100,16.468140,Background,Europe,0,39
1861,XKO,Kosovo (under UNSCR 1244/99),2019,Prizren,20.741500,42.215800,22.627710,Background,Europe,0,39
1862,XKO,Kosovo (under UNSCR 1244/99),2019,Muhaxher,21.092640,42.393470,16.436100,Background,Europe,0,39
1863,XKO,Kosovo (under UNSCR 1244/99),2019,StagovÃ«,21.273800,42.284000,20.366410,Background,Europe,0,39


In [30]:
euro.to_csv('euro_pm25.csv', 
             columns=['Year', 'CountryName', 'CityReverseGeocoded', 'Longitude', 'Latitude',
                      'PM25', 'is_urban', 'country'],
                     quoting=csv.QUOTE_NONNUMERIC, index=False)

In [ ]:
na_slice = world[world['ISO3'].isin(['CAN','USA','MEX'])]
north_america = na_slice.copy(deep=True).reset_index(drop=True)
north_america['country'] = north_america['ISO3'].astype('category').cat.codes + 1

In [ ]:
north_america.head(3)

In [ ]:
north_america.tail(3)

In [ ]:
north_america.shape

In [ ]:
north_america.to_csv('north_america_pm25.csv', 
             columns=['Year', 'CityReverseGeocoded', 'Longitude', 'Latitude', 
                      'PM25', 'is_urban', 'country'], 
                     quoting=csv.QUOTE_NONNUMERIC, index=False)